# **Reporte de Datos**
---

In [1]:
# Librerias
import os
import pandas as pd
import requests
import logging
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Funcion para crear el objeto de creación de logs
def create_logger():
  logging.basicConfig(level = logging.INFO, format = '%(asctime)s - %(levenname)s - %(message)s')
  logger = logging.getLogger('Logger')
  logger.info('Logger creado')
  return logger

In [3]:
# Función de consumo a la base de datos de Firebase para cargar el DataSet
def download_firebase(url, logger):
  logger.info("Extrayendo el archivo desde Firebase")
  df = None
  try:
    df = pd.read_csv(url)
    logger.info("Archivo cargado")
  except requests.exceptions.RequestException as e:
    logger.info(f"Error al descargar el archivo CSV: {e}")
  except pd.errors.EmptyDataError:
    logger.info("El archivo CSV está vacío.")
  except Exception as e:
    logger.info(f"Ocurrió un error inesperado: {e}")
  return df

In [4]:
# Cargar DataSet
url = 'https://firebasestorage.googleapis.com/v0/b/personalwp-8822c.appspot.com/o/diabetes_prediction_dataset.csv?alt=media&token=4d70d154-c3d0-4fa0-a3aa-9b9972dd3b95'
logger = create_logger()
df = download_firebase(url, logger)

## **Resumen de Calidad de los Datos**

### **Selección de Datos**

In [5]:
# Valores atípicos en 'bmi'
seventy_fifth = df['bmi'].quantile(0.75)
twenty_fifth = df['bmi'].quantile(0.25)
iqr = seventy_fifth - twenty_fifth
upper = seventy_fifth + (10 * iqr)
outliers_bmi_upper = df[(df['bmi'] > upper)]

In [6]:
# Eliminación de valores atípicos en 'bmi'
df = pd.merge(df, outliers_bmi_upper, indicator = True, how = 'outer').query('_merge == "left_only"').drop('_merge', axis = 1)

In [7]:
# Eliminación de valores duplicados
df = df.drop_duplicates(keep = "first")

In [8]:
# Información del DataSet
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 96143 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   gender               96143 non-null  object 
 1   age                  96143 non-null  float64
 2   hypertension         96143 non-null  int64  
 3   heart_disease        96143 non-null  int64  
 4   smoking_history      96143 non-null  object 
 5   bmi                  96143 non-null  float64
 6   HbA1c_level          96143 non-null  float64
 7   blood_glucose_level  96143 non-null  int64  
 8   diabetes             96143 non-null  int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 7.3+ MB


**Universidad Nacional de Colombia** - *Facultad de Ingeniería*